In [1]:
# Import dependencies
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json
import pandas as pd
from pandas.io import sql
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score   
from sklearn.linear_model import LogisticRegression
import numpy
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.metrics import r2_score
from consts import *
from sklearn.svm import SVC
%matplotlib inline

In [2]:
# Connecting to Postgres instance
engine = create_engine(CREATE_ENGINE_STR)

In [3]:
# Printing info for table names 
print (engine.table_names())

['committees', 'candidates', 'education', 'six_state_donations', 'donations', 'classifying_results', 'health_metrics', 'agg_county_votes', 'birth_death_rate', 'postal_codes', 'res_lr', 'agg_county_donors', 'pres_votes_6t', 'unemployment', 'res_log']


In [4]:
#Get all donation records for a single state and return it in a dataframe
def donor_state_query(state):
    #Run queries to get all donation records from the states into dfs
    #select_sql = f'select * from {TABLE_SIX_STATE_DONATIONS} where "STATE"=\'{state.upper()}\''
    select_sql = f'select * from {TABLE_SIX_STATE_DONATIONS} where "STATE"=\'{state.upper()}\' LIMIT 10000'
    donor_df = pd.read_sql_query(select_sql, con=engine)
    return donor_df

In [5]:
committee_df = pd.read_sql_query('select * from "committees"', con=engine)

In [6]:
donor_df = donor_state_query("FL")

In [7]:
donor_df = committee_df.merge(donor_df, left_on='CMTE_ID', right_on='CMTE_ID')

In [8]:
donor_df = merge_cmtid_party(donor_df)

In [9]:
y_params = ["party"]
X_params = ["CITY", "STATE", "ZIP", "EMPLOYER", "OCCUPATION", "TRANSACTION_AMT"]

In [10]:
donor_df.tail()

,CMTE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_DSGN,CMTE_TP,...,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,party
55884,C00575795,HILLARY FOR AMERICA,"JONES, ELIZABETH",P.O. BOX 5256,None,NEW YORK,NY,101855256,P,P,...,RETIRED,10212016,25,None,C13443050,1148953,None,None,4021720171371204030,democrat
55885,C00575795,HILLARY FOR AMERICA,"JONES, ELIZABETH",P.O. BOX 5256,None,NEW YORK,NY,101855256,P,P,...,RETIRED,10222016,37,None,C13464560,1148953,None,None,4021720171371204470,democrat
55886,C00575795,HILLARY FOR AMERICA,"JONES, ELIZABETH",P.O. BOX 5256,None,NEW YORK,NY,101855256,P,P,...,SALES,11072016,25,None,C15733104,1148953,None,None,4021720171371415993,democrat
55887,C00592774,MURPHY VICTORY FUND,BRIAN FOUCART,1050 17TH STREET NW SUITE 590,None,WASHINGTON,DC,20036,J,N,...,EXECUTIVE,12062016,100,C00401224,VPF6TMA7JT1,1144430,None,* EARMARKED CONTRIBUTION: SEE BELOW,4020320171369516248,other
55888,C00592774,MURPHY VICTORY FUND,BRIAN FOUCART,1050 17TH STREET NW SUITE 590,None,WASHINGTON,DC,20036,J,N,...,EXECUTIVE,12062016,100,C00401224,VPF6TMA7JT1,1144430,None,* EARMARKED CONTRIBUTION: SEE BELOW,4020320171369516248,other
